In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import itertools

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용할 장치: {device}")

print("데이터를 로드하는 중...")
df = pd.read_csv("item.csv")

def combine_text(row):
    return f"{row['title']} [SEP] {row['short_description']}"

print("제목과 설명을 하나의 텍스트로 결합하는 중...")
df['combined_text'] = df.apply(combine_text, axis=1)

# 차원 축소를 위한 선형 레이어 추가
class BertWithProjection(nn.Module):
    def __init__(self, output_dim=256):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.projection = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled = outputs.pooler_output
        return self.projection(pooled)

print("BERT 모델과 토크나이저를 로드하는 중...")
bert_model = BertWithProjection(output_dim=256).to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_combined_embedding(text, tokenizer, model, max_length=512):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding=True,
        truncation=True, 
        max_length=max_length
    ).to(device)
    
    with torch.no_grad():
        embedding = model(**inputs)
    return embedding.squeeze().cpu().numpy()

print("설명+제목 텍스트 임베딩 생성 중...")
df['combined_embedding'] = [
    get_combined_embedding(text, tokenizer, bert_model) 
    for text in tqdm(df['combined_text'], desc="설명+제목 임베딩 생성")
]

print("태그와 카테고리를 인코딩하는 중...")
# category 처리: 콤마로 분리하여 좌우 공백 제거 후 리스트 생성
df['categories_list'] = df['categories'].fillna('').apply(lambda x: [s.strip() for s in x.split(',')] if x != '' else [])
# 고유 category 목록 생성
unique_categories = sorted(set(itertools.chain.from_iterable(df['categories_list'])))
cat2idx = {cat: i for i, cat in enumerate(unique_categories)}
print("Unique category count:", len(unique_categories))
# 각 row마다 category 인덱스 리스트 생성
df['encoded_categories'] = df['categories_list'].apply(lambda cats: [cat2idx[cat] for cat in cats])

# tag 처리: tag 정보도 같은 방식으로 변환 (열 이름이 tags_sum라고 가정)
df['tags_list'] = df['tags_sum'].fillna('').apply(lambda x: [s.strip() for s in x.split(',')] if x != '' else [])
unique_tags = sorted(set(itertools.chain.from_iterable(df['tags_list'])))
tag2idx = {tag: i for i, tag in enumerate(unique_tags)}
print("Unique tag count:", len(unique_tags))
df['encoded_tags'] = df['tags_list'].apply(lambda tags: [tag2idx[tag] for tag in tags])

class EmbeddingLayer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    
    def forward(self, x):
        return self.embedding(x)
    
# 임베딩 레이어 초기화: 고유 클래스 개수를 이용
tag_embedding_layer = EmbeddingLayer(num_embeddings=len(unique_tags), embedding_dim=64).to(device)
category_embedding_layer = EmbeddingLayer(num_embeddings=len(unique_categories), embedding_dim=32).to(device)

def get_embedding_from_indices(indices, layer):
    if not indices:
        return torch.zeros(layer.embedding.embedding_dim, device=device)
    idx_tensor = torch.tensor(indices, dtype=torch.long, device=device)
    emb = layer(idx_tensor)
    return emb.mean(dim=0)

df['tag_embedding'] = df['encoded_tags'].apply(lambda inds: get_embedding_from_indices(inds, tag_embedding_layer).cpu().detach().numpy())
df['category_embedding'] = df['encoded_categories'].apply(lambda inds: get_embedding_from_indices(inds, category_embedding_layer).cpu().detach().numpy())

# item_id와 임베딩 매핑 생성
embeddings_df = pd.DataFrame({
    'item_id': df['item_id'].values,
    'combined_embedding': list(df['combined_embedding']),
    'tag_embedding': list(df['tag_embedding']),
    'category_embedding': list(df['category_embedding'])
})

print("최종 임베딩이 완료되었습니다!")
print(f"최종 임베딩 shape: {embeddings_df.shape}")
print("\n결과 예시:")
print(embeddings_df.head())

# 최종 임베딩 저장
print("최종 임베딩을 저장하는 중...")
embeddings_df.to_csv("item_embeddings.csv", index=False)
embeddings_df.to_pickle("item_embeddings.pkl")
print("최종 임베딩이 저장되었습니다!")